In [1]:
# !python -m spacy download en_core_web_md
# !python -m spacy download en_core_web_sm
# !pip install spacy[transformers]
# pip install fuzzywuzzy
# pip install python-Levenshtein
# !pip install openpyxl
# !pip install python-certifi-win32
# !pip install spacy
# pip install ruamel.yaml
# pip install pyarrow
#pip install swifter
#pip install pandarallel [--upgrade] [--user]
#pip install pandarallel
# pip install pandarallel --upgrade
#pip install dask
#pip install "dask[complete]" 
#pip install joblib

In [2]:
import swifter
import spacy
import ssl
from spacy import displacy
import pandas as pd
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from sklearn.metrics import accuracy_score
from spacy.matcher import Matcher
from spacy.pipeline import EntityRuler

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy import process, fuzz

from pandarallel import pandarallel
import dask.dataframe as dd


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tqdm import tqdm
tqdm.pandas()
#pandarallel.initialize(progress_bar=True)

import warnings
warnings.filterwarnings('ignore')


In [3]:
df = pd.read_parquet('C:/Users/Olimpio.Dasilva/Desktop/MACHINE LEARNING/DATA/reporting_ncr_all_transactions_202302131044.parquet')


In [4]:
df.head(5)

,fi_id,account_id,transaction_id,amount,currency,txn_desc1,txn_desc2,region,transaction_timestamp,merchant_id,merchant_name,payment_agent,payment_agent_id,extract_date,filename,name
0,DI1004,Adb7cee870873375e9b71f97b49bb47f3,T8c3444845ea837d98f6492fbfeb996b1,300.0,USD,External Withdrawal TJMAXX,MASTERCAR NEEDHAM BANK BP BILL PAYMT,MT,2023-02-08,12897.0,TJ MAXX,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
1,DI1004,A0d0f7d433a6f3cf4815fc5f7f77bbf73,Td0c9a05199c034f282105c258edf4c90,42.5,USD,POS Withdrawal (FIP) SUNOCO,*63 SUNOCO *63 FRAMINGHAM MA(8631),None,2023-02-08,12696.0,SUNOCO,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
2,DI1004,Ac9fb869c66b83cb7a6f0d84e5f37d60a,Tf9ce6b86a4e1379eb600c9af4c1ae90e,250.0,USD,External Withdrawal BARCLAYCARD,US CREDITCARD,None,2023-02-07,NaN,Unknow Merchant,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
3,DI1004,Ade85fffa5da73362bd6b0522721820a3,T4f1445aa5fd536f09d13bd6918c057fb,42.5,USD,ATM Withdrawal (FIP) SUNSHINE,LLC-60 464 MAIN ST WEST SPRINGFI MA(5681),None,2023-02-08,NaN,Unknow Merchant,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
4,DI1004,A1f3390a72bf035a48a21fce7d72df58a,T7be6870fc08e30c48fd78836885adf12,114.5,USD,POS Withdrawal (FIP) NNT,DESTINATION 735 PROVIDENCE HWY DEDHAM MA(7169),None,2023-02-11,NaN,Unknow Merchant,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank


In [5]:
print('Total number of lines and columns on the original dataset is: ', df.shape)

Total number of lines and columns on the original dataset is:  (2280287, 16)


1. Dataset with Unknow Merchants

In [6]:
df_unknown_merchs = df.copy(deep=True)
df_unknown_merchs.shape

(2280287, 16)

2. Dataset with Known Merchants

In [7]:
df_known_merchs = df_unknown_merchs.copy(deep=True)
df_known_merchs = df_known_merchs[df_unknown_merchs["merchant_name"].str.contains("Unknow Merchant") == False]
df_known_merchs.shape

(1133623, 16)

In [8]:
#Creating new columns CONCAT_COLS, AFS_MECHANT_NAMES, NER_RAW_MODEL

pd.set_option('display.max_colwidth', None)


df2 = df_known_merchs[['fi_id', 'account_id', 'transaction_id','txn_desc1','txn_desc2','merchant_name']].copy(deep=True)
df2["CONCAT_COLS"] = df['txn_desc1'].astype(str) +" %%%%% "+ df['txn_desc2'].astype(str)

df2['NER_RAW_MODEL'] = pd.Series(dtype='string')

df2['AFS_MECHANT_NAMES']= df2['merchant_name'].str.lower()

#df2 = df2.head(1000)

df2.shape

(1133623, 9)

In [9]:
df2.head(5)

,fi_id,account_id,transaction_id,txn_desc1,txn_desc2,merchant_name,CONCAT_COLS,NER_RAW_MODEL,AFS_MECHANT_NAMES
0,DI1004,Adb7cee870873375e9b71f97b49bb47f3,T8c3444845ea837d98f6492fbfeb996b1,External Withdrawal TJMAXX,MASTERCAR NEEDHAM BANK BP BILL PAYMT,TJ MAXX,External Withdrawal TJMAXX %%%%% MASTERCAR NEEDHAM BANK BP BILL PAYMT,<NA>,tj maxx
1,DI1004,A0d0f7d433a6f3cf4815fc5f7f77bbf73,Td0c9a05199c034f282105c258edf4c90,POS Withdrawal (FIP) SUNOCO,*63 SUNOCO *63 FRAMINGHAM MA(8631),SUNOCO,POS Withdrawal (FIP) SUNOCO %%%%% *63 SUNOCO *63 FRAMINGHAM MA(8631),<NA>,sunoco
5,DI1004,A47e41d8c2c6c3fb2860572bfc15dd11a,T644d8104fbda356aaa98203bb5723f4e,POS Withdrawal (FIP) KOHLS 0575,1 WORCESTER RD FRAMINGHAM MA(8806),KOHL'S,POS Withdrawal (FIP) KOHLS 0575 %%%%% 1 WORCESTER RD FRAMINGHAM MA(8806),<NA>,kohl's
7,DI1004,Ac5f923f551923ff99f9b1983b7641f63,Tce633b3d9de130218e493c10b9fbe09a,POS Withdrawal (FIS),AMAZON.COM*HE3H AMAZON.COM SEATTLE WA(6966),AMAZON.COM,POS Withdrawal (FIS) %%%%% AMAZON.COM*HE3H AMAZON.COM SEATTLE WA(6966),<NA>,amazon.com
12,DI1004,Ac5f923f551923ff99f9b1983b7641f63,T1704c488eec6363fa58d5b2b3a770c7b,POS Withdrawal (FIS),AMAZON.COM*HR0D AMAZON.COM SEATTLE WA(6966),AMAZON.COM,POS Withdrawal (FIS) %%%%% AMAZON.COM*HR0D AMAZON.COM SEATTLE WA(6966),<NA>,amazon.com


2.1. NER model application

In [10]:
def raw_model(text:str):
    NER = spacy.load("en_core_web_sm")    
    text_ner = NER(text)
    r_orgs = []
    for word in text_ner.ents:
        if word.label_ == "ORG":
            r_orgs.append(word.text)
    return ' '.join(r_orgs) if r_orgs else 'NA'

In [ ]:
df2['NER_RAW_MODEL'] = df2['CONCAT_COLS'].swifter.apply(raw_model)


In [11]:
df2['CONCAT_COLS']= df2['CONCAT_COLS'].str.lower().str.replace(r'\s{2,}','').str.replace(r"[,;@#?!&$*]+\ *", " ")
df2['NER_RAW_MODEL']= df2['NER_RAW_MODEL'].str.lower().str.replace(' ','').fillna('-')

df2.head(5)

,fi_id,account_id,transaction_id,txn_desc1,txn_desc2,merchant_name,CONCAT_COLS,NER_RAW_MODEL,AFS_MECHANT_NAMES
0,DI1004,Adb7cee870873375e9b71f97b49bb47f3,T8c3444845ea837d98f6492fbfeb996b1,External Withdrawal TJMAXX,MASTERCAR NEEDHAM BANK BP BILL PAYMT,TJ MAXX,external withdrawal tjmaxx %%%%% mastercar needham bank bp bill paymt,-,tj maxx
1,DI1004,A0d0f7d433a6f3cf4815fc5f7f77bbf73,Td0c9a05199c034f282105c258edf4c90,POS Withdrawal (FIP) SUNOCO,*63 SUNOCO *63 FRAMINGHAM MA(8631),SUNOCO,pos withdrawal (fip) sunoco %%%%% 63 sunoco 63 framingham ma(8631),-,sunoco
5,DI1004,A47e41d8c2c6c3fb2860572bfc15dd11a,T644d8104fbda356aaa98203bb5723f4e,POS Withdrawal (FIP) KOHLS 0575,1 WORCESTER RD FRAMINGHAM MA(8806),KOHL'S,pos withdrawal (fip) kohls 0575 %%%%% 1 worcester rd framingham ma(8806),-,kohl's
7,DI1004,Ac5f923f551923ff99f9b1983b7641f63,Tce633b3d9de130218e493c10b9fbe09a,POS Withdrawal (FIS),AMAZON.COM*HE3H AMAZON.COM SEATTLE WA(6966),AMAZON.COM,pos withdrawal (fis) %%%%% amazon.com he3h amazon.com seattle wa(6966),-,amazon.com
12,DI1004,Ac5f923f551923ff99f9b1983b7641f63,T1704c488eec6363fa58d5b2b3a770c7b,POS Withdrawal (FIS),AMAZON.COM*HR0D AMAZON.COM SEATTLE WA(6966),AMAZON.COM,pos withdrawal (fis) %%%%% amazon.com hr0d amazon.com seattle wa(6966),-,amazon.com


In [12]:
df2.nunique()

fi_id                    204
account_id            129930
transaction_id       1086638
txn_desc1             132917
txn_desc2             655042
merchant_name           1793
CONCAT_COLS           745566
NER_RAW_MODEL              1
AFS_MECHANT_NAMES       1793
dtype: int64

In [13]:
(df2.AFS_MECHANT_NAMES == df2.NER_RAW_MODEL).value_counts()

False    1133623
dtype: Int64

In [14]:
# Getting number of total merchant names
a = df2.AFS_MECHANT_NAMES
aunique = a.unique()
dist_list = pd.DataFrame(aunique)
print('The total number of distinct merchants is: ', len(dist_list))

The total number of distinct merchants is:  1793


In [15]:
df3 = df2[['transaction_id','CONCAT_COLS','AFS_MECHANT_NAMES', 'NER_RAW_MODEL']].copy(deep=True)

In [16]:
#Creating a list of patterns - with all the merchant names (with and without spaces, quotes, .com, etc.)

org = df3['AFS_MECHANT_NAMES']

#List of Entities and Patterns
org_list = org.tolist()

a = [ x.strip() for x in org_list] # list comprehension - all merchant names from original org_list
a_space =list(filter(lambda x: ' ' in x, a)) # filtering all names w spaces
a_nospace = [x.replace(' ', '') for x in a_space] # replacing spaces w ''

a_quo =list(filter(lambda x: "'" in x, a)) # filtering all names w quotes
a_noquuo = [x.replace("'", '') for x in a_quo] # replacing quotes w ''

a_com =list(filter(lambda x: '.com' in x, a)) # filtering all names w .com
a_nocom = [x.replace('.com', '') for x in a_com] # replacing .com w ''

a_dash =list(filter(lambda x: '-' in x, a)) # filtering all names w dash
a_nodash = [x.replace('-', '') for x in a_com] # replacing dash w ''

new_lis = a + a_nospace + a_nocom + a_noquuo + a_nodash # add instead of dis_list


# Creating a list of patterns with all the variations from new_lis
patterns = []
i = 0
for item in new_lis:
    patterns.append({"label": 'ORG', "pattern": new_lis[i]})
    i+=1

#print(patterns)



In [17]:
# Function to find merchant names in a string - using an NER model with the previously created patterns

def find_org(text:str):
    
    #Build upon the spaCy Model
    NER2 = spacy.load("en_core_web_sm")
    
    #Create the EntityRuler
    ruler = NER2.add_pipe("entity_ruler", config={'validate': True})
     
    ruler.add_patterns(patterns)
    doc_ner = NER2(text)
    
    # useless strings
    junk = ['fip', 'ca(2810','fis','xx4386', '#185','*','hr0d','k50ms3hv','xx9510','xx4710','xx4706','xx0322','he34i8pz0','xx3840','pd1387']
    
    #extract entities
    orgs = []
    for ent in doc_ner.ents:
        if ((ent.label_ == "ORG") and (ent.text not in junk)):
            orgs.append(ent.text)
    orgs = list(set(orgs))
    return ' '.join(orgs) if orgs else 'NA'


In [18]:
df3['PREDICTIONS'] = df3.CONCAT_COLS.swifter.apply(find_org)
df3.head(5)

MemoryError: Unable to allocate 1.41 MiB for an array with shape (1287, 288) and data type float32

In [ ]:
# Function to compare results - True/False

def check_names(a, b):
    if ((a in b) or (b in a)):
        return True
    elif (fuzz.ratio(a, b) > 92):
        return True
    else:
        return False

In [ ]:
#New column CHECK for True/False results

df3['CHECK'] = df3.apply(lambda x: check_names(x.AFS_MECHANT_NAMES, x.PREDICTIONS), axis=1)
df3.head(5)

In [ ]:

df3['SIMILARITY'] = pd.Series(dtype='string')

df3['SIMILARITY'] = df3.apply(lambda x: fuzz.ratio(x['AFS_MECHANT_NAMES'], x['PREDICTIONS']), axis=1)

#df3['SIMILARITY'] = df3.apply(lambda x: fuzz.token_sort_ratio(x['AFS_MECHANT_NAMES'], x['PREDICTIONS']), axis=1)

#df3['SIMILARITY'] = df3.apply(lambda x: fuzz.WRatio(x['AFS_MECHANT_NAMES'], x['PREDICTIONS']), axis=1)

df3.head(10)

In [ ]:
df3.shape

In [ ]:
(df3.AFS_MECHANT_NAMES == df3.PREDICTIONS).value_counts()

In [ ]:
# Checking Accuracy
print('Accuracy: {0:0.9f}'.format(accuracy_score(df3.PREDICTIONS, df3.AFS_MECHANT_NAMES)))